In [8]:
import numpy as np

import pycollocation

I based the following on lecture notes on [investment with adjustment costs](http://eml.berkeley.edu/~webfac/gourinchas/e202a_f14/Notes_Investment_pog.pdf) from UC Berkeley.

## Non-renewable energy sector

Sector combines capital, $K_{NR}$, and fossil fuels, $F$, to produce energy $E_{NR}$ via standard Cobb-Douglas production function.

$$ E_{NR}(t) = \psi K_{NR}(t)^{\beta}F(t)^{1-\beta} \tag{1} $$

### Profits

Sector profits are...

$$ \Pi_{NR}(t) = P_E E_{NR}(t) - P_F F(t) - P_K\big(1 + C(I_{NR}, K_{NR})\big)I_{NR}(t) \tag{2} $$

...where $P_E$, $P_F$, and $P_K$ are, respectively, the prices of energy, fossil fuels, and capital goods; finally, $C(I, K)$ represents the *percentage* increase in costs required to install one unit of capital. In what follows I use the following formulation for the adjustment costs function.

$$ C(I_{NR}(t), K_{NR}(t)) = \frac{\phi}{2}\bigg(\frac{I_{NR}(t)}{K_{NR}(t)} - \delta\bigg)^2 \tag{3} $$

Note that this formulation implies that percentage adjustment costs are *convex* in the level of investment; since adjustment costs depend on the ratio of investment to installed capital, percentage adjustment costs scale up with the level of capital. 

### Objective

Want to solve...

$$ \max_{I_{NR}(t), F(t)} \int_0^{\infty} e^{-rt} \Pi_{NR}(t) dt \tag{4} $$

...subject to the following constraint...

$$ \dot{K}_{NR}(t) = I_{NR}(t) - \delta K_{NR}(t). \tag{5} $$

### Solution

Start by setting up the current value Hamiltonian...

$$ H(F(t), I(t), \lambda(t)) \equiv P_E \psi K_{NR}(t)^{\beta}F(t)^{1-\beta} - P_F F(t) - P_K\bigg(1 + \frac{\phi}{2}\bigg(\frac{I_{NR}(t)}{K_{NR}(t)} - \delta\bigg)^2\bigg)I_{NR}(t) + \lambda(t)\big[I_{NR}(t) - \delta K_{NR}(t)\big] \tag{6} $$ 

...first order conditions are...

\begin{align}
    \frac{\partial H}{\partial F} = 0 \implies& P_E \psi (1 - \beta) K_{NR}(t)^{\beta}F(t)^{-\beta} = P_F \tag{7} \\
    \frac{\partial H}{\partial I} = 0 \implies& 1 + \frac{\phi}{2}\bigg(\frac{I_{NR}(t)}{K_{NR}(t)} - \delta\bigg)^2 + \phi\bigg(\frac{I_{NR}(t)}{K_{NR}(t)} - \delta\bigg)\frac{I_{NR}(t)}{K_{NR}(t)} = \frac{\lambda(t)}{P_K} \tag{8} \\
    \frac{\partial H}{\partial K} = -\bigg(\dot{\lambda}(t) - r\lambda(t)\bigg) \implies& r\lambda(t) - P_E \psi \beta K_{NR}(t)^{\beta-1}F(t)^{1-\beta} - P_K\phi\bigg(\frac{I_{NR}(t)}{K_{NR}(t)} - \delta\bigg)\bigg(\frac{I_{NR}(t)}{K_{NR}(t)}\bigg)^2 + \delta\lambda(t) = \dot{\lambda}(t) \tag{9}
\end{align}

...now solve equation 7 for $F(t)$ and equation 8 for $I_{NR}(t)$ and substitute both into equation 9...

\begin{align}
    F(t) =& \bigg(\psi(1-\beta)\frac{P_E}{P_F}\bigg)^{\frac{1}{\beta}}K_{NR}(t) \tag{10} \\
    \frac{\lambda(t)}{P_K} =& 1 + \frac{\phi}{2}\bigg(\frac{I_{NR}(t)}{K_{NR}(t)} - \delta\bigg)^2 + \phi\bigg(\frac{I_{NR}(t)}{K_{NR}(t)} - \delta\bigg)\frac{I_{NR}(t)}{K_{NR}(t)} \\
    \frac{\lambda(t)}{P_K} =& 1 + \frac{\phi}{2}\bigg(\frac{I_{NR}(t)}{K_{NR}(t)} - \delta\bigg)^2 + \phi\bigg(\frac{I_{NR}(t)}{K_{NR}(t)}\bigg)^2 - \delta\phi\bigg(\frac{I_{NR}(t)}{K_{NR}(t)}\bigg)
\end{align}


In [28]:
def investment_demand(t, K, q, delta, phi, r, **params):
    """Non-renewable energy sector investment in physical capital."""
    I = (((np.exp(r * t) * q - 1) / phi) - delta) * K
    return I


def fossil_fuel_demand(K, q, beta, energy_price, fossil_fuel_price, psi, **params):
    """Non-renewable energy sector demand for fossil fuels."""
    relative_price = energy_price / fossil_fuel_price
    F = (psi * (1 - beta) * relative_price)**(1 / beta) * K
    return F


def K_dot(t, K, q, delta, **params):
    """Equation of motion for non-renewable sector physical capital."""
    I = investment_demand(t, K, q, delta, **params)
    return I - delta * K


def q_dot(t, K, q, delta, r, **params):
    """Equation of motion for Tobin's q."""
    I = investment_demand(t, K, q, delta, r=r, **params)
    F = fossil_fuel_demand(K, q, **params)
    return (q - delta) - np.exp(-r * t) * net_value_marginal_product_capital(F, I, K, delta, **params)
    

def net_value_marginal_product_capital(F, I, K, delta, **params):
    nvmp = (value_marginal_product_capital(F, K, **params) - marginal_capital_adjustment_costs(I, K, delta, **params))
    return nvmp


def value_marginal_product_capital(F, K, energy_price, **params):
    """
    Value marginal product (VMP) of capital is the marginal increase in revenue
    generated by an additional unit of installed capital.
    
    """
    vmp = energy_price * marginal_product_capital(F, K, **params)
    return vmp


def marginal_product_capital(F, K, beta, **params):
    """Marginal product of physical capital."""
    return beta / capital_output_ratio(F, K, beta, **params)


def capital_output_ratio(F, K, beta, **params):
    return K / output(F, K, beta, **params)


def output(F, K, beta, psi, **params):
    """Non-renewable energy sector output."""
    return psi * K**beta * F**(1 - beta)


def marginal_capital_adjustment_costs(I, K, delta, phi, **params):
    return capital_adjustment_costs(I, K, delta, phi, **params) - phi * ((I / K) - delta) * (I / K)


def capital_adjustment_costs(I, K, delta, phi, **params):
    """Per unit costs of adjusting stock of physical capital."""
    return (phi / 2) * ((I / K) - delta)**2


In [17]:
def model(t, K, q, **params):
    return [K_dot(t, K, q, **params), q_dot(t, K, q, **params)]


def initial_condition(t, K, q, K0, **params):
    return [K - K0]


def terminal_condition(t, K, q):
    """In steady state"""
    return [q]

In [18]:
non_renewable_params = {'beta': 0.67, 'psi': 1, 'r': 0.09, 'fossil_fuel_price': 1, 'phi': 0.2,
                        'energy_price': 1.0, 'delta': 0.04, 'K0': 1.0}

In [29]:
model(0.0, 1.0, 1.0, **non_renewable_params)

[-0.080000000000000002, 0.57191670213540013]

In [31]:
bvp = pycollocation.problems.TwoPointBVP(bcs_lower=initial_condition,
                                         bcs_upper=terminal_condition,
                                         number_bcs_lower=1,
                                         number_odes=2,
                                         params=non_renewable_params,
                                         rhs=model,
                                         )

In [32]:
def initial_mesh(t, T, num, problem):
    # compute equilibrium values
    cstar = equilibrium_consumption(**problem.params)
    kstar = equilibrium_capital(**problem.params)
    ystar = cobb_douglas_output(kstar, **problem.params)

    # create the mesh for capital
    ts = np.linspace(t, T, num)
    k0 = problem.params['K0'] / (problem.params['A0'] * problem.params['N0'])
    ks = kstar - (kstar - k0) * np.exp(-ts)

    # create the mesh for consumption
    s = 1 - (cstar / ystar)
    y0 = cobb_douglas_output(k0, **problem.params)
    c0 = (1 - s) * y0
    cs = cstar - (cstar - c0) * np.exp(-ts)

    return ts, ks, cs

In [ ]:
polynomial_basis = pycollocation.basis_functions.PolynomialBasis()
solver = pycollocation.solvers.Solver(polynomial_basis)

boundary_points = (0, 200)
ts, ks, cs = initial_mesh(*boundary_points, num=1000, problem=standard_ramsey_bvp)

basis_kwargs = {'kind': 'Chebyshev', 'domain': boundary_points, 'degree': 25}
k_poly = polynomial_basis.fit(ts, ks, **basis_kwargs)
c_poly = polynomial_basis.fit(ts, cs, **basis_kwargs)
initial_coefs = np.hstack([k_poly.coef, c_poly.coef])
nodes = polynomial_basis.roots(**basis_kwargs)

solution = solver.solve(basis_kwargs, boundary_points, initial_coefs,
                        nodes, standard_ramsey_bvp)